This is the prototype for making the book recommendation system with collaborative filtering. The dataset can be found from: https://www.kaggle.com/datasets/saurabhbagchi/books-dataset/data

In [61]:
#Import libraries
import pandas as pd
import numpy as np

from scipy.sparse import csr_matrix

In [24]:
book_file = pd.read_csv("books_data/books.csv", sep=";", encoding='latin-1', on_bad_lines='skip')

C:\Users\Sim\AppData\Local\Temp\ipykernel_4088\2935500988.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  book_file = pd.read_csv("books_data/books.csv", sep=";", encoding='latin-1', on_bad_lines='skip')


In [25]:
book_file.head()

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...


In [26]:
user_file = pd.read_csv("books_data/users.csv", sep=";", encoding='latin-1', on_bad_lines='skip')

In [27]:
user_file.head()

,User-ID,Location,Age
0,1,"nyc, new york, usa",NaN
1,2,"stockton, california, usa",18.0
2,3,"moscow, yukon territory, russia",NaN
3,4,"porto, v.n.gaia, portugal",17.0
4,5,"farnborough, hants, united kingdom",NaN


In [28]:
rating_file = pd.read_csv("books_data/ratings.csv", sep=";", encoding='latin-1', on_bad_lines='skip')

In [29]:
rating_file.head()

,User-ID,ISBN,Book-Rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6


In [30]:
rating_file["User-ID"].value_counts()

User-ID
11676     13602
198711     7550
153662     6109
98391      5891
35859      5850
          ...  
116180        1
116166        1
116154        1
116137        1
276723        1
Name: count, Length: 105283, dtype: int64

In [31]:
#Since we need users with alot of ratings we filter out the ones less than 150 (random value)
filterrating = rating_file["User-ID"].value_counts() > 50

In [36]:
filternum = filterrating[filterrating].index

In [38]:
rating_file = rating_file[rating_file["User-ID"].isin(filternum)]

In [41]:
book_file.head()

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...


In [42]:
#merge ratings and book files
bookratings = rating_file.merge(book_file, on="ISBN")

In [45]:
bookratings.head()

,User-ID,ISBN,Book-Rating,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,276847,0446364193,0,Along Came a Spider (Alex Cross Novels),James Patterson,1993,Warner Books,http://images.amazon.com/images/P/0446364193.0...,http://images.amazon.com/images/P/0446364193.0...,http://images.amazon.com/images/P/0446364193.0...
1,276847,3379015180,0,Schlafes Bruder,Robert Schneider,1994,"Reclam, Leipzig",http://images.amazon.com/images/P/3379015180.0...,http://images.amazon.com/images/P/3379015180.0...,http://images.amazon.com/images/P/3379015180.0...
2,276847,3404145909,8,Der Stein der Kelten.,Michael Phillips,2001,LÃ?Â¼bbe,http://images.amazon.com/images/P/3404145909.0...,http://images.amazon.com/images/P/3404145909.0...,http://images.amazon.com/images/P/3404145909.0...
3,276847,3404148576,8,Nordermoor,Arnaldur Indridason,2003,LÃ?Â¼bbe,http://images.amazon.com/images/P/3404148576.0...,http://images.amazon.com/images/P/3404148576.0...,http://images.amazon.com/images/P/3404148576.0...
4,276847,3404921178,7,Nur der Tod ist ohne Makel.,Ann Granger,2002,LÃ?Â¼bbe,http://images.amazon.com/images/P/3404921178.0...,http://images.amazon.com/images/P/3404921178.0...,http://images.amazon.com/images/P/3404921178.0...


In [48]:
#show how many ratings each book has
numbookratings = bookratings.groupby("Book-Title")["Book-Rating"].count().reset_index()

In [49]:
numbookratings

,Book-Title,Book-Rating
0,A Light in the Storm: The Civil War Diary of ...,4
1,Always Have Popsicles,1
2,Apple Magic (The Collector's series),1
3,"Ask Lily (Young Women of Faith: Lily Series, ...",1
4,Beyond IBM: Leadership Marketing and Finance ...,1
...,...,...
198798,Ã?Â?berraschung am Valentinstag.,1
198799,Ã?Â?lpiraten.,1
198800,Ã?Â?rger mit Produkt X. Roman.,3
198801,Ã?Â?stlich der Berge.,1


In [51]:
ratings = bookratings.merge(numbookratings, on="Book-Title")

In [52]:
ratings.head()

,User-ID,ISBN,Book-Rating_x,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L,Book-Rating_y
0,276847,0446364193,0,Along Came a Spider (Alex Cross Novels),James Patterson,1993,Warner Books,http://images.amazon.com/images/P/0446364193.0...,http://images.amazon.com/images/P/0446364193.0...,http://images.amazon.com/images/P/0446364193.0...,195
1,276847,3379015180,0,Schlafes Bruder,Robert Schneider,1994,"Reclam, Leipzig",http://images.amazon.com/images/P/3379015180.0...,http://images.amazon.com/images/P/3379015180.0...,http://images.amazon.com/images/P/3379015180.0...,10
2,276847,3404145909,8,Der Stein der Kelten.,Michael Phillips,2001,LÃ?Â¼bbe,http://images.amazon.com/images/P/3404145909.0...,http://images.amazon.com/images/P/3404145909.0...,http://images.amazon.com/images/P/3404145909.0...,3
3,276847,3404148576,8,Nordermoor,Arnaldur Indridason,2003,LÃ?Â¼bbe,http://images.amazon.com/images/P/3404148576.0...,http://images.amazon.com/images/P/3404148576.0...,http://images.amazon.com/images/P/3404148576.0...,6
4,276847,3404921178,7,Nur der Tod ist ohne Makel.,Ann Granger,2002,LÃ?Â¼bbe,http://images.amazon.com/images/P/3404921178.0...,http://images.amazon.com/images/P/3404921178.0...,http://images.amazon.com/images/P/3404921178.0...,5


In [53]:
ratings = ratings[ratings["Book-Rating_y"] > 30]

In [54]:
ratings.shape

(187555, 11)

In [55]:
ratings.drop_duplicates(["User-ID","Book-Title"], inplace=True)

C:\Users\Sim\AppData\Local\Temp\ipykernel_4088\225137787.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ratings.drop_duplicates(["User-ID","Book-Title"], inplace=True)


In [57]:
#Now we need to create a table with each rating for each book with user-ID
table = ratings.pivot_table(columns="User-ID", index="Book-Title", values ="Book-Rating_x")

In [58]:
table

User-ID,183,243,254,507,626,638,643,741,882,929,...,277928,277965,278026,278137,278144,278188,278418,278582,278633,278843
Book-Title,,,,,,,,,,,,,,,,,,,,,
10 Lb. Penalty,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
101 Dalmatians,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN
16 Lighthouse Road,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN
1984,NaN,NaN,9.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1st to Die: A Novel,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Zlata's Diary: A Child's Life in Sarajevo,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Zoya,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"\O\"" Is for Outlaw""",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [59]:
table.fillna(0, inplace=True)

In [62]:
#Need to turn to compressed matrix
matrix = csr_matrix(table)